# Exercise 1: 

<div class="line-block"><code>1   It does not print the fruit at the correct index, why is the returned result wrong?</code><br />
<code>2   How could this be fixed?</code></div>

### Solution: 
 The issue with the code is that all the fruits all in a set and the problem with a set is that it is not in an order. So, for returning the correct fruit name for each fruit id, we should use a List instead of a Set. (As shown below) 

In [ ]:
def id_to_fruit(fruit_id: int, fruits: list) -> str:
    
    if fruit_id < 0 or fruit_id >= len(fruits):
        raise ValueError(f"Fruit with id {fruit_id} does not exist")
    return fruits[fruit_id]

fruits = ["apple", "orange", "melon", "kiwi", "strawberry"]
name1 = id_to_fruit(1, fruits)
name3 = id_to_fruit(3, fruits)
name4 = id_to_fruit(4, fruits)
print(name1,name3,name4)

# Exercise 2:

<div class="line-block"><code>1   Can you spot the obvious error?</code><br />
<code>2   After fixing the obvious error it is still wrong, how can this be fixed?</code></div>

### Solution: 
The obvious error is:

 ```coords[:, 0], coords[:, 1], ... = coords[:, 1], coords[:, 1], ... ```

it should be:

 ```coords[:, 0], coords[:, 1], ... = coords[:, 1], coords[:, 0], ....```

Also, by printing out the "swapped_croods", we would get all the Y coordinates as all the X coordinates, which is wrong. (regarding that we only needed them to be flipped.)
And as a solution, we can copy each coords on the other side, so that we wouldn't have the same columns of Y(s) where the X column sits. (as shown below)

In [ ]:
import numpy as np
def swap(coords: np.ndarray):
    coords[:, 0], coords[:, 1], coords[:, 2], coords[:, 3] = coords[:, 1].copy(), coords[:, 0].copy(), coords[:, 3].copy(), coords[:, 2].copy()
    return coords

coords = np.array([[10, 5, 15, 6, 0],
                   [11, 3, 13, 6, 0],
                   [5, 3, 13, 6, 1],
                   [4, 4, 13, 6, 1],
                   [6, 5, 13, 16, 1]])
swapped_coords = swap(coords)

print(swapped_coords)


# Exercies 3:

`1. For some reason the plot is not showing correctly, can you find out what is going wrong?`

`2. How could this be fixed?`

### Solution: 

The x and y values are being interchanged in the plot. The CSV file lists precision in the first column and recall in the second column. However, the plot function is erroneously plotting recall on the x-axis and precision on the y-axis.

To rectify this, we should interchange the columns when plotting. we can use the `pandas` library to streamline CSV file reading for improved efficiency.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

def plot_data(csv_file_path: str):
    # load data
    results = []
    with open(csv_file_path) as result_csv:
        csv_reader = csv.reader(result_csv, delimiter=',')
        next(csv_reader)
        for row in csv_reader:
            results.append(row)
        results = np.stack(results)
    plt.plot(results[:, 0], results[:, 1])
    plt.ylim([-0.05, 1.05])
    plt.xlim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

# Exerecise 4: 

`1   Changing the batch_size from 32 to 64 triggers the structural bug.`

`2. Can you also spot the cosmetic bug?`


### Solution:

The bug trigged by changing the batch size to 64 is caused by a mismatch between the size of the discriminator output and the label sizes. The discriminator output size is based on the batch size, but the label sizes are hardcoded to 32. to fix this change these two lines: 

 ``` real_samples_labels = torch.ones((batch_size, 1)).to(device=device) ```
 ``` generated_samples_labels = torch.zeros((batch_size, 1)).to(device=device)``` 

into these:
```real_samples_labels = torch.ones(real_samples.size(0), 1).to(device=device)```
```generated_samples_labels = torch.zeros(latent_space_samples.size(0), 1).to(device=device)```


When retrying the MNIST download, it's advisable to utilize the updated URL from the ```MNIST.resources``` property to ensure adaptability to future changes.
The training loop currently displays generated images every batch, which is redundant and resource-intensive. Consider reducing the frequency to, for example, every 10 batches.
Refactoring the try/except block for downloading MNIST into a separate utility function would enhance the cleanliness of the training code.
Incorporating high-level comments to elucidate the overall flow and architectural decisions would enhance code comprehensibility.
Integrating validation or testing code to assess the GAN's performance would align with best practices in machine learning.
Adding the iteration number to the loss prints would provide valuable context for analysis.

In [ ]:
import torch
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchvision
NoneType = type(None)
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from PIL import Image
import torchvision.transforms.functional as TF
from torchvision.models import vgg11
from torchvision.models import mobilenet_v2
import torchvision.transforms as transforms
import time

# You can copy this code to your personal pipeline project or execute it here.
def train_gan(batch_size: int = 32, num_epochs: int = 100, device: str = "cuda:0" if torch.cuda.is_available() else "cpu"):
    # Add/adjust code.

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    try:
        train_set = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=transform)
    except:
        print("Failed to download MNIST, retrying with different URL")
        # see: https://github.com/pytorch/vision/blob/master/torchvision/datasets/mnist.py
        torchvision.datasets.MNIST.resources = [
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz',
             'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz',
             'd53e105ee54ea40749a09fcbcd1e9432'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz',
             '9fb629c4189551a2d022fa330f9573f3'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz',
             'ec29112dd5afa0611ce80d1b7f02629c')
        ]
        train_set = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

    # example data
    real_samples, mnist_labels = next(iter(train_loader))

    fig = plt.figure()
    for i in range(16):
        sub = fig.add_subplot(4, 4, 1 + i)
        sub.imshow(real_samples[i].reshape(28, 28), cmap="gray_r")
        sub.axis('off')

    fig.tight_layout()
    fig.suptitle("Real images")
    display(fig)

    time.sleep(5)

    # Set up training
    discriminator = Discriminator().to(device)
    generator = Generator().to(device)
    lr = 0.0001
    loss_function = nn.BCELoss()
    optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
    optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

    # train
    for epoch in range(num_epochs):
        for n, (real_samples, mnist_labels) in enumerate(train_loader):

            # Data for training the discriminator
            real_samples = real_samples.to(device=device)
            real_samples_labels = torch.ones(real_samples.size(0), 1).to(device=device)
            latent_space_samples = torch.randn((batch_size, 100)).to(device=device)
            generated_samples = generator(latent_space_samples)
            generated_samples_labels = torch.zeros(latent_space_samples.size(0), 1).to(device=device)
            all_samples = torch.cat((real_samples, generated_samples))
            all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))

            # Training the discriminator
            discriminator.zero_grad()
            output_discriminator = discriminator(all_samples)
            loss_discriminator = loss_function(output_discriminator, all_samples_labels)
            loss_discriminator.backward()
            optimizer_discriminator.step()

            # Data for training the generator
            latent_space_samples = torch.randn((batch_size, 100)).to(device=device)

            # Training the generator
            generator.zero_grad()
            generated_samples = generator(latent_space_samples)
            output_discriminator_generated = discriminator(generated_samples)
            loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
            loss_generator.backward()
            optimizer_generator.step()

            # Show loss and samples generated
            if n == batch_size - 1:
                name = f"Generate images\n Epoch: {epoch} Loss D.: {loss_discriminator:.2f} Loss G.: {loss_generator:.2f}"
                generated_samples = generated_samples.detach().cpu().numpy()
                fig = plt.figure()
                for i in range(16):
                    sub = fig.add_subplot(4, 4, 1 + i)
                    sub.imshow(generated_samples[i].reshape(28, 28), cmap="gray_r")
                    sub.axis('off')
                fig.suptitle(name)
                fig.tight_layout()
                clear_output(wait=False)
                display(fig)

